#### Translating mteb/sts17-crosslingual-sts

We will use OpenAI's `GPT-4o-mini` to machinely translate `en-en` pairs to include `ua-ua` and `ua-en` pairs. For additional research, the benchmark may also be translated into languages that are close or similar to Ukrainian.
